<a href="https://colab.research.google.com/github/sejhig2/Project_mask_healthy/blob/main/makeModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 사용 전 숙지사항
- 아래 코드는 colab 에서 작성됐습니다.
- 드라이브는 개인 구글 드라이브를 사용해야합니다.
- 이미지, label 데이터는 [링크](https://drive.google.com/file/d/1VPGPzV8KrZFKsQJq7jL3ukiKsFzy2Eh8/view?usp=sharing) 걸어놓거나, 압축파일에 있는 것을 사용하면 됩니다.

- 구글 드라이브 마운트 해주기


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
# 코드 실행 후 뜨는 인터넷 창에서 로그인하고 코드 복사해서 아래의 공란에 붙여넣어줘야한다.

- 이미지와 라벨의 경로를 알아야한다. -> 경로를 전달해서 훈련할 때 사용해야한다.

In [ ]:
from glob import glob
img_path = "이미지 폴더 경로"+"/*"
annotation_path = "라벨 폴더 경로"+"/*"

img_list = glob(img_path)
anno_list = glob(annotation_path)
print(len(img_list))
print(len(anno_list))

- 코랩에 yolo v5를 가져온다.

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

- yolo v5가 필요로 하는 패키지가 requirments.txt 파일에 담겨있으니 설치하자.

In [ ]:
%cd /content/yolov5/
!pip install -r requirements.txt

- 기본 정보. 파일들의 경로와 클래스가 data.yaml 파일에 담겨있다. yaml 파일을 열어보고 수정할게 있다면 수정해야한다.
- 만약 data.yaml 파일을 가져올 수 없다면 메모장에다가 만들어야 한다.
- 기본적인 form은 압축파일과 함께 첨부

In [ ]:
%cat /content/data.yaml

- 데이터가 한 곳에 있고 아직 훈련, 검정, 테스트 용으로 나눠져있지 않으니 나눠주자.

In [ ]:
from sklearn.model_selection import train_test_split
train_img_list, val_img_list = train_test_split( img_list, test_size = 0.1 )
print(len(train_img_list), len(val_img_list))

- txt파일 만들어주기 (훈련, 검증이미지의 경로를 넣어줄 것이다.)

In [ ]:
train_txt_path = " aa"
train_txt_path = train_txt_path + "/train.txt" 
print(train_txt_path)

In [ ]:
train_txt_path = "편의를 위해서 dataset이 있는 폴더로 지정하면 편리rain_txt_path = "편의를 위해서 dataset이 있는 폴더로 지정하면 편리."
train_txt_path = train_txt_path + "/train.txt" 
val_txt_path = "편의를 위해서 dataset이 있는 폴더로 지정하면 편리."
vali_txt_path = train_txt_path + "/val.txt" 

with open(train_txt_path,"w") as f :
    f.write("\n".join(train_img_list)+"\n")
    with open(vali_txt_path, "w") as f:
        f.write("\n".join(val_img_list)+"\n")."
train_txt_path = train_txt_path + "/train.txt" 
val_txt_path = "편의를 위해서 dataset이 있는 폴더로 지정하면 편리."
vali_txt_path = train_txt_path + "/val.txt" 

with open(train_txt_path,"w") as f :
    f.write("\n".join(train_img_list)+"\n")
with open(vali_txt_path, "w") as f:
    f.write("\n".join(val_img_list)+"\n")

- yaml 파일의 훈련, 검증 이미지의 경로를 담은 txt파일을 위치를 write

    data.yaml 파일이 생성되지 않는 경우가 있다. 첨부파일을 확인해서 열어주자.

In [ ]:
data_yaml_path = "data.yaml파일의 경로 넣어주세요."
import yaml
with open(data_yaml_path, "r") as f :
    data = yaml.load(f)
print(data)

data["train"] = train_txt_path
data["val"] = vali_txt_path

with open(data_yaml_path, "w") as f :
    yaml.dump( data, f )
print(data)

### 훈련 진행 
-  --data 뒤에 data.yaml 파일의 경로를 입력
-  --cfg 뒤에 yolov5의 cfg파일을 골라서 넣어주자. s,m,l,x 중에 하나일 것
-  --weights 는 yolov5에서 다운받아 오는 것.
-  --name 훈련을 마친 후 저장할 폴더의 이름

In [ ]:
%cd /content/yolov5/

!python train.py --img 416 --batch 16 --epochs 250 --data /content/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name withAnnotation_yolov5s_results

### 텐서보드를 열어서 훈련한 모델의 성능을 수치적으로 알아보자.
- --logdir 훈련을 마친 후 모델이 담겨있는 폴더의 경로를 넣어줘야한다. 폴더에 여러개의 모델이 담겨있다면 한꺼번에 그래프들을 그려준다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/yolov5/runs/train/withAnnotation_yolov5s_results